In [ ]:
load('quandles-alpha.sage')

In [2]:
# Debuggable analyze_pd_sage for Sage with UPDATED PD convention:
# PD tuple (a,b,c,d) meaning:
#   a = incoming under (under_enter)
#   b = outgoing over  (over_exit)
#   c = outgoing under (under_exit)
#   d = incoming over  (over_enter)
#
# Tangents:
#   under strand follows a -> c  (t_under = p_c - p_a)
#   over  strand follows d -> b  (t_over  = p_b - p_d)
#
import math
from collections import defaultdict

# ---------------- geometry helpers ----------------
def vector_sub(p, q): return (p[0]-q[0], p[1]-q[1])
def det2(u, v): return u[0]*v[1] - u[1]*v[0]
def normalize(v):
    n = math.hypot(v[0], v[1])
    return (v[0]/n, v[1]/n) if n else (0.0, 0.0)
def safe_tangent(p_from, p_to):
    v = vector_sub(p_to, p_from)
    if abs(v[0]) < 1e-12 and abs(v[1]) < 1e-12:
        v = (1e-6, 0.0)
    return normalize(v)

# ---------------- place arc points ----------------
def place_arc_points(arcs, radius=1.0):
    """Place each original arc label at a distinct point on a circle (deterministic jitter)."""
    arcs_sorted = list(sorted(arcs))
    m = max(1, len(arcs_sorted))
    pts = {}
    for i, a in enumerate(arcs_sorted):
        th = 2 * math.pi * i / m
        j = ((int(a) * 37) % 1000 - 500) / 100000.0
        th += j
        pts[a] = (radius * math.cos(th), radius * math.sin(th))
    return pts

# ---------------- compute signs from original PD (NEW convention) ----------------
def compute_signs_from_pd(pd, debug=False):
    """
    Input: pd list of 4-tuples (a,b,c,d) using the NEW convention:
           a = incoming under, b = outgoing over, c = outgoing under, d = incoming over.
    Returns (crossings, arc_set, occurrences) where crossings is dict(index->info).
    """
    # Make sure pd is a list of 4-tuples
    pd = [tuple(t) for t in pd]
    # collect original arcs and occurrences
    occurrences = defaultdict(list)   # arc_label -> list of (crossing_index, role, position_in_tuple)
    for i, tup in enumerate(pd, start=1):
        if len(tup) != 4:
            raise ValueError(f"PD entry {i} is not length 4: {tup}")
        a,b,c,d = tup
        # NEW mapping of roles (per your convention)
        occurrences[a].append((i, 'under_enter', 0))   # incoming under
        occurrences[b].append((i, 'over_exit',   1))   # outgoing over
        occurrences[c].append((i, 'under_exit',  2))   # outgoing under
        occurrences[d].append((i, 'over_enter',  3))   # incoming over
    arc_set = sorted(occurrences.keys())

    if debug:
        print("Raw PD (as provided):")
        for i,tup in enumerate(pd, start=1):
            print(f" {i}: {tup}")
        print("\nArc occurrences (arc_label -> occurrences):")
        for arc in arc_set:
            print(f"  {arc}: {occurrences[arc]}")

    # Basic sanity check: each arc should appear exactly twice (for a single component)
    bad = [arc for arc, occ in occurrences.items() if len(occ) != 2]
    if bad and debug:
        print("\nWarning: some arcs do not occur exactly twice (this may indicate a non-standard PD or multi-component link):", bad)

    # place points using original arc labels
    pts = place_arc_points(arc_set)
    crossings = {}
    tol = 1e-9
    for i, (a,b,c,d) in enumerate(pd, start=1):
        p_a, p_b, p_c, p_d = pts[a], pts[b], pts[c], pts[d]
        # NEW tangents per your convention:
        #   under: a -> c    (incoming under to outgoing under)
        #   over:  d -> b    (incoming over to outgoing over)
        t_under = safe_tangent(p_a, p_c)  # careful: safe_tangent(from, to) -> vector to - from
        t_over  = safe_tangent(p_d, p_b)
        # Note: your previous code used safe_tangent(p_b, p_a) etc. Here we follow a->c and d->b
        # compute signed determinant
        det = det2(t_under, t_over)
        sign = None
        if det > tol:
            sign = 1
        elif det < -tol:
            sign = -1
        else:
            # small deterministic perturb attempt (try to break degeneracy)
            sign = 0
            for trial in range(6):
                pts_trial = dict(pts)
                for label in (a,b,c,d):
                    x,y = pts_trial[label]
                    eps = (0.00025 + 0.00005*trial) * (((int(label)*31) % 7) - 3)
                    th = math.atan2(y,x) + eps
                    r = math.hypot(x,y)
                    pts_trial[label] = (r * math.cos(th), r * math.sin(th))
                t_under = safe_tangent(pts_trial[a], pts_trial[c])  # a -> c
                t_over  = safe_tangent(pts_trial[d], pts_trial[b])  # d -> b
                det_trial = det2(t_under, t_over)
                if abs(det_trial) > tol:
                    det = det_trial
                    sign = 1 if det > 0 else -1
                    pts = pts_trial
                    break
            if sign == 0:
                # final fallback (avoid None)
                sign = 1
        crossings[i] = {
            'pd_tuple_original': (a,b,c,d),
            'under_original': (a,c),   # under arc is a -> c
            'over_original': (d,b),    # over  arc is d -> b
            'det_original': float(det),
            'sign': int(sign),
            'sign_label': '+' if sign==1 else '-'
        }
    return crossings, arc_set, occurrences

# ---------------- merge arcs by over-arc connectivity (NEW: union d and b) ----------------
def merge_arcs_from_pd(pd, debug=False):
    arc_set = set()
    for a,b,c,d in pd:
        arc_set.update([a,b,c,d])
    parent = {x:x for x in arc_set}
    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x
    def union(x,y):
        rx, ry = find(x), find(y)
        if rx==ry:
            return
        # deterministically choose smaller rep
        if rx < ry: parent[ry] = rx
        else: parent[rx] = ry
    # union over-arcs based on NEW convention: over-arc endpoints are (d,b)
    for a,b,c,d in pd:
        union(d, b)   # merge incoming-over (d) with outgoing-over (b)
    groups = {}
    for x in sorted(arc_set):
        r = find(x)
        groups.setdefault(r, []).append(x)
    # produce stable gid assignment in sorted order of representative
    reps = sorted(groups.keys())
    rep_to_gid = {rep: (i+1) for i,rep in enumerate(reps)}
    gid_to_members = {rep_to_gid[r]: groups[r] for r in reps}
    label_to_group = {label: rep_to_gid[find(label)] for label in arc_set}
    pd_merged = [(label_to_group[a], label_to_group[b], label_to_group[c], label_to_group[d]) for a,b,c,d in pd]
    if debug:
        print("\nMerge map (original -> group):")
        for lab in sorted(label_to_group):
            print(" ", lab, "-> g", label_to_group[lab])
        print("\nMerged PD (tuples):")
        for i,t in enumerate(pd_merged, start=1):
            print(" ", i, t)
    return {'groups': gid_to_members, 'label_to_group': label_to_group, 'pd_merged': pd_merged, 'reduced_arcs': sorted(gid_to_members.keys())}

# ---------------- high-level analyzer (signs first, merge second) ----------------
def analyze_pd_sage_debug(pd, debug=True):
    """
    Computes signs on original PD (with NEW tuple convention), then merges arcs for naming.
    Returns kinfo dict and prints debug info if debug=True.
    """
    if debug:
        print("=== Running analyze_pd_sage_debug (NEW PD convention) ===")
    crossings_raw, original_arcs, occurrences = compute_signs_from_pd(pd, debug=debug)
    if debug:
        print("\nCrossings (original, computed signs):")
        for i,info in crossings_raw.items():
            print(f" {i}: orig_pd={info['pd_tuple_original']}, sign={info['sign_label']}, det={info['det_original']:.6g}")

    merged = merge_arcs_from_pd(pd, debug=debug)
    pd_merged = merged['pd_merged']
    merged_arcs = merged['reduced_arcs']

    # Build final crossing records attaching merged tuple but keep original sign
    crossings_final = {}
    arc_to_crossings = {g:[] for g in merged_arcs}
    for i, info in crossings_raw.items():
        a,b,c,d = info['pd_tuple_original']
        a_m, b_m, c_m, d_m = merged['label_to_group'][a], merged['label_to_group'][b], merged['label_to_group'][c], merged['label_to_group'][d]
        crossings_final[i] = {
            'pd_tuple_original': (a,b,c,d),
            'pd_tuple_merged': (a_m,b_m,c_m,d_m),
            'det_original': info['det_original'],
            'sign': info['sign'],
            'sign_label': info['sign_label']
        }
        for lab in (a_m,b_m,c_m,d_m):
            arc_to_crossings.setdefault(lab, []).append(i)

    kinfo = {
        'original_arcs': original_arcs,
        'merged': merged,
        'arcs': merged_arcs,
        'crossings': crossings_final,
        'arc_to_crossings': arc_to_crossings,
        'occurrences': occurrences
    }

    if debug:
        print("\nFinal summary (merged tuples + signs):")
        for ci,inf in sorted(kinfo['crossings'].items()):
            print(f" {ci}: pd_merged={inf['pd_tuple_merged']}, sign={inf['sign_label']}, det(original)={inf['det_original']:.6g}")
        print("\nArc -> crossings (merged labels):")
        for a,xs in sorted(kinfo['arc_to_crossings'].items()):
            print(" arc", a, ":", xs)
        print("\nTotal merged arcs:", len(kinfo['arcs']))
    return kinfo

# ---------------- Example you can replace pd_example with your PD ----------------
# Example PD for a trefoil-like input (you can paste your PD here)
pd_example = [(5,2,0,3), (3,0,4,1), (1,4,2,5)]
kinfo = analyze_pd_sage_debug(pd_example, debug=True)
print("kinfo keys:", sorted(kinfo.keys()))


=== Running analyze_pd_sage_debug (NEW PD convention) ===
Raw PD (as provided):
 1: (5, 2, 0, 3)
 2: (3, 0, 4, 1)
 3: (1, 4, 2, 5)

Arc occurrences (arc_label -> occurrences):
  0: [(1, 'under_exit', 2), (2, 'over_exit', 1)]
  1: [(2, 'over_enter', 3), (3, 'under_enter', 0)]
  2: [(1, 'over_exit', 1), (3, 'under_exit', 2)]
  3: [(1, 'over_enter', 3), (2, 'under_enter', 0)]
  4: [(2, 'under_exit', 2), (3, 'over_exit', 1)]
  5: [(1, 'under_enter', 0), (3, 'over_enter', 3)]

Crossings (original, computed signs):
 1: orig_pd=(5, 2, 0, 3), sign=+, det=0.000875
 2: orig_pd=(3, 0, 4, 1), sign=-, det=-0.00136
 3: orig_pd=(1, 4, 2, 5), sign=+, det=0.000485

Merge map (original -> group):
  0 -> g 1
  1 -> g 1
  2 -> g 2
  3 -> g 2
  4 -> g 3
  5 -> g 3

Merged PD (tuples):
  1 (3, 2, 1, 2)
  2 (2, 1, 3, 1)
  3 (1, 3, 2, 3)

Final summary (merged tuples + signs):
 1: pd_merged=(3, 2, 1, 2), sign=+, det(original)=0.000875
 2: pd_merged=(2, 1, 3, 1), sign=-, det(original)=-0.00136
 3: pd_merged=(1

In [3]:
pd_example = [(5,2,0,3), (3,0,4,1), (1,4,2,5)]
kinfo = analyze_pd_sage_debug(pd_example, debug=True)

=== Running analyze_pd_sage_debug (NEW PD convention) ===
Raw PD (as provided):
 1: (5, 2, 0, 3)
 2: (3, 0, 4, 1)
 3: (1, 4, 2, 5)

Arc occurrences (arc_label -> occurrences):
  0: [(1, 'under_exit', 2), (2, 'over_exit', 1)]
  1: [(2, 'over_enter', 3), (3, 'under_enter', 0)]
  2: [(1, 'over_exit', 1), (3, 'under_exit', 2)]
  3: [(1, 'over_enter', 3), (2, 'under_enter', 0)]
  4: [(2, 'under_exit', 2), (3, 'over_exit', 1)]
  5: [(1, 'under_enter', 0), (3, 'over_enter', 3)]

Crossings (original, computed signs):
 1: orig_pd=(5, 2, 0, 3), sign=+, det=0.000875
 2: orig_pd=(3, 0, 4, 1), sign=-, det=-0.00136
 3: orig_pd=(1, 4, 2, 5), sign=+, det=0.000485

Merge map (original -> group):
  0 -> g 1
  1 -> g 1
  2 -> g 2
  3 -> g 2
  4 -> g 3
  5 -> g 3

Merged PD (tuples):
  1 (3, 2, 1, 2)
  2 (2, 1, 3, 1)
  3 (1, 3, 2, 3)

Final summary (merged tuples + signs):
 1: pd_merged=(3, 2, 1, 2), sign=+, det(original)=0.000875
 2: pd_merged=(2, 1, 3, 1), sign=-, det(original)=-0.00136
 3: pd_merged=(1

In [5]:
import snappy
M = snappy.Link('3_1')
pd = M.PD_code()       # ← works for both knots and links
print(pd)

ModuleNotFoundError: No module named 'snappy'

In [6]:
# Sage: build quandle matrix equations from kinfo (your analyze_pd_sage result)
from sage.all import PolynomialRing, matrix, QQ, var
from sage.matrix.constructor import Matrix
import math

# ---- algebra helpers (2x2) ----
def adjugate_2x2(M):
    a,b,c,d = M[0,0], M[0,1], M[1,0], M[1,1]
    return Matrix(M.base_ring(), [[d, -b], [-c, a]])

def det_2x2(M):
    a,b,c,d = M[0,0], M[0,1], M[1,0], M[1,1]
    return a*d - b*c

# ---- build symbolic 2x2 matrices for k generators ----
def build_variable_matrices(k):
    # variables: g1_11,g1_12,g1_21,g1_22, g2_11, ...
    names = []
    for i in range(1, k+1):
        names += [f"g{i}_11", f"g{i}_12", f"g{i}_21", f"g{i}_22"]
    R = PolynomialRing(QQ, names)
    syms = list(R.gens())
    mats = {}
    for i in range(1, k+1):
        idx = (i-1)*4
        a,b,c,d = syms[idx], syms[idx+1], syms[idx+2], syms[idx+3]
        mats[i] = matrix(R, [[a,b],[c,d]])
    return R, mats

# ---- optional: create parametric template matrices (like your quandleElement(alpha,m)) ----
def quandle_template_matrix(alpha, m, R):
    # Use the form you used: [[(alpha*m+1)/m, alpha], [(-alpha*m + m^2 -1)/m, -alpha + m]]
    # create elements in ring R by coercion
    am = alpha * m
    return matrix(R, [[ (am + 1) / m, alpha ],
                      [ (-am + m**2 - 1) / m, -alpha + m ]])

# ---- build polynomial equations from kinfo ----
def build_polys_from_kinfo(kinfo, use_template=False, template_map=None, debug=False):
    """
    kinfo: output from analyze_pd_sage_debug (must contain 'arcs' and 'crossings').
    use_template: if True, template_map must map merged-arc-id -> (alpha_symbol, m_symbol)
                  or you can pass a single global mapping for all arcs.
    template_map: dict {arc_id: (alpha_name, m_name)} or None.
    Returns: R (PolynomialRing), mats (arc_id->2x2 matrix over R), pols (list of polynomials)
    """
    merged_arcs = sorted(kinfo['arcs'])
    k = len(merged_arcs)
    # build base ring and variable matrices
    R_var, var_mats = build_variable_matrices(k)
    # We'll optionally build a template ring if templates are requested
    if use_template:
        # collect all alpha/m symbol names used in template_map
        symbols = set()
        if template_map is None:
            raise ValueError("use_template=True requires template_map")
        for v in template_map.values():
            if isinstance(v, tuple):
                symbols.update([v[0], v[1]])
            else:
                raise ValueError("template_map values must be (alpha_name, m_name)")
        # build extended polynomial ring including template symbols and variable symbols
        ext_names = list(symbols) + list(R_var.variable_names())
        R = PolynomialRing(QQ, ext_names)
        # map variable symbols into R
        name_to_gen_R = {name: R.gen(i) for i,name in enumerate(R.variable_names())}
        # reconstruct variable mats in R (so we have both variable mats and template mats in same ring)
        var_mats_R = {}
        for i in range(1, k+1):
            a_name = f"g{i}_11"; b_name = f"g{i}_12"; c_name = f"g{i}_21"; d_name = f"g{i}_22"
            a = name_to_gen_R[a_name]; b = name_to_gen_R[b_name]; c = name_to_gen_R[c_name]; d = name_to_gen_R[d_name]
            var_mats_R[i] = matrix(R, [[a,b],[c,d]])
        # build template matrices (coerce symbols)
        mats = {}
        for idx, arc in enumerate(merged_arcs, start=1):
            if arc in template_map:
                alpha_name, m_name = template_map[arc]
                alpha = name_to_gen_R[alpha_name]; m = name_to_gen_R[m_name]
                mats[arc] = quandle_template_matrix(alpha, m, R)
            else:
                # default use symbolic variable matrix for this arc
                mats[arc] = var_mats_R[idx]
    else:
        # no template: use purely variable matrices, over R_var
        R = R_var
        mats = {arc: var_mats[i+1] for i,arc in enumerate(merged_arcs)}
    # Build polynomials: for each crossing (use mapping y=a, z=c, x=d)
    pols = []
    for ci, info in sorted(kinfo['crossings'].items()):
        a_m, b_m, c_m, d_m = info['pd_tuple_merged']   # merged labels
        sign = info['sign']
        # mapping per your quandle relation:
        # y (enter under) = a_m
        # z (exit under)  = c_m
        # x (upper strand) = d_m   (incoming over)
        G_y = mats[a_m]
        G_z = mats[c_m]
        G_x = mats[d_m]
        detGx = det_2x2(G_x)
        adjGx = adjugate_2x2(G_x)
        if sign == 1:
            # positive: G_z = G_x^{-1} * G_y * G_x  => detGx * G_z - adjGx * G_y * G_x == 0
            L = G_z * detGx
            Rmat = adjGx * G_y * G_x
        else:
            # negative: G_y = G_x^{-1} * G_z * G_x  => detGx * G_y - adjGx * G_z * G_x == 0
            L = G_y * detGx
            Rmat = adjGx * G_z * G_x
        diff = L - Rmat
        # add 4 scalar polynomial components
        pols.extend([diff[0,0], diff[0,1], diff[1,0], diff[1,1]])
        if debug:
            print("crossing", ci, "sign", sign, "detGx:", detGx)
    return R, mats, pols

# ---- compute groebner basis helper ----
def groebner_basis_from_polys(R, pols, order='lex', debug=False):
    I = R.ideal(pols)
    if debug:
        print("Ideal created; computing Groebner basis (this can take time)...")
    G = I.groebner_basis()
    return G

# ---------------- Example usage (use your kinfo) ----------------
# If you have kinfo in your notebook (result of analyze_pd_sage_debug), leave it; otherwise this example will create a small placeholder:
try:
    kinfo  # if kinfo exists, use it
except NameError:
    print("No kinfo found; building a small example (trefoil-like merged arcs).")
    kinfo = {
        'arcs': [1,2,3],
        'merged': {'groups': {1:[0,3], 2:[1,4], 3:[2,5]}},
        'crossings': {
            1: {'pd_tuple_merged': (3,3,1,1), 'sign': 1},
            2: {'pd_tuple_merged': (1,1,2,2), 'sign': 1},
            3: {'pd_tuple_merged': (2,2,3,3), 'sign': 1}
        }
    }

# Build polynomials from kinfo (no template)
R, mats, pols = build_polys_from_kinfo(kinfo, use_template=False, template_map=None, debug=True)
print("Ring:", R)
print("Number of polynomial equations:", len(pols))

# Compute Groebner basis (may take time)
G = groebner_basis_from_polys(R, pols, order='lex', debug=True)
print("Groebner basis (first 6 entries):")
for p in G[:6]:
    print(" ", p)

# ---------------- Optional: using your alpha,m template for some arcs ----------------
# Example template_map: map merged-arc-id -> (alpha_name, m_name)
# e.g. template_map = {1:('alpha1','m1'), 2:('alpha2','m2')}
# Then call:
# R2, mats2, pols2 = build_polys_from_kinfo(kinfo, use_template=True, template_map=template_map)
# G2 = groebner_basis_from_polys(R2, pols2)
# and then you can substitute m1=1 etc:
# G2_sub = [poly.subs({R2('m1'):1}) for poly in G2]


ModuleNotFoundError: No module named 'sage'